# TinnyLlma + QLoRA Finetunning



In [2]:
# @title 安装Unsloth
!pip install unsloth accelerate transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [19]:
# @title Environment Setup and Validation
import torch
from unsloth import FastLanguageModel
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# Print Environment Information
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch版本: 2.6.0+cu124
CUDA可用: True
CUDA版本: 12.4
GPU: Tesla T4
GPU内存: 15.83 GB


In [20]:
# CHECK CUDA VERSION
!nvcc --version
print("\n")

# CHECK SYSTEM INFORMATION AND GPU
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


Sat Mar  8 03:08:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0     

In [5]:
# Clone Project Repository for necessary files
!git clone https://github.com/michaelearncoding/MSE718FinalProject-FinetuneAnalysisForLLMTechnique.git

Cloning into 'MSE718FinalProject-FinetuneAnalysisForLLMTechnique'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 65 (from 1)
Receiving objects: 100% (66/66), 74.84 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Updating files: 100% (40/40), done.


In [6]:
%cd /content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique


/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique


In [15]:
import random
# Create 2K Sample Subset
if os.path.exists("llm-peft-compare/data/alpaca_train.jsonl"):
    # Read all lines
    with open("llm-peft-compare/data/alpaca_train.jsonl", "r") as f:
        lines = f.readlines()

    # Randomly select 2000 lines
    random.seed(42)
    sample = random.sample(lines, min(2000, len(lines)))

    # Write to subset
    with open("llm-peft-compare/data/alpaca_train_2k.jsonl", "w") as f:
        f.writelines(sample)

    print(f"✅ Created 2K data subset, {len(sample)} data")
else:
    print("❌ JSONL file does not exist")

✅ 创建了2K数据子集，共2000条数据


In [16]:
!pwd
# content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique


In [18]:
# @title 数据准备
import json
from datasets import Dataset

# load 2K dataset and convert to Unsloth format
with open("llm-peft-compare/data/alpaca_train_2k.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Convert to instruction format
formatted_data = []
for item in data:
    instruction = item["instruction"]
    input_text = item.get("input", "")
    output = item["output"]

    # build prompt
    if input_text:
        prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "
    else:
        prompt = f"Instruction: {instruction}\nOutput: "

    formatted_data.append({"prompt": prompt, "response": output})

# Create HF dataset
dataset = Dataset.from_list(formatted_data)
print(f"Data set prepared, {len(dataset)} samples")

数据集准备完成，共2000个样本


In [21]:
# 创建必要的目录结构
!mkdir -p llm-peft-compare/data
!mkdir -p llm-peft-compare/models
!mkdir -p llm-peft-compare/results/figures
!mkdir -p llm-peft-compare/results/model_comparison
!mkdir -p llm-peft-compare/results/raw_data

In [7]:
# # 准备数据集
# %cd llm-peft-compare

# # 检查是否已有数据，如果没有则下载
# if not os.path.exists("data/alpaca_train.jsonl"):
#     !wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json -O data/alpaca_data.json
#     # 转换为JSONL格式
#     !python -c "import json; data = json.load(open('data/alpaca_data.json')); [print(json.dumps(item)) for item in data]" > data/alpaca_train.jsonl

# # 创建5K数据子集
# if not os.path.exists("data/alpaca_train_5k.jsonl"):
#     !python scripts/create_subset_data.py --input data/alpaca_train.jsonl --output data/alpaca_train_5k.jsonl --size 5000

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare


In [25]:
%cd /content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare


In [26]:
!pwd

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare


## 开始训练: Phi-2 + QLoRA - 使用Unsloth进行微调:


接下来我们将执行`fast_train.sh`脚本，使用'small'模型尺寸（Phi-2）和'qlora'方法进行训练。

In [33]:
# @title 正确加载JSONL数据
import json
import os
from datasets import Dataset

# 确保数据目录存在
os.makedirs("data", exist_ok=True)

# 数据路径
data_path = "data/alpaca_train.jsonl"

# 确认文件是否存在
if not os.path.exists(data_path):
    print(f"文件不存在: {data_path}")
    print("尝试下载原始数据...")
    !wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json -O data/alpaca_data.json

    # 转换为JSONL
    if os.path.exists("data/alpaca_data.json"):
        with open("data/alpaca_data.json", "r") as f:
            data_list = json.load(f)

        with open(data_path, "w") as f:
            for item in data_list:
                f.write(json.dumps(item) + "\n")

        print(f"✅ 已创建JSONL文件: {data_path}")

#load data with strict format
data = []
with open(data_path, "r") as f:
    for i, line in enumerate(f):
        try:
            item = json.loads(line.strip())
            if isinstance(item, dict) and "instruction" in item and "output" in item:
                data.append(item)
            else:
                print(f"跳过第{i+1}行: 格式不正确")
        except json.JSONDecodeError:
            print(f"跳过第{i+1}行: JSON解析错误")

print(f"成功加载了{len(data)}条有效数据")

# Only take the first 2000 as training data
training_data = data[:2000]
print(f"Use the first 2000 data for training")

# Convert to Unsloth required format
formatted_data = []
for item in training_data:
    instruction = item["instruction"]
    input_text = item.get("input", "")
    output = item["output"]

    # build prompt
    if input_text:
        prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "
    else:
        prompt = f"Instruction: {instruction}\nOutput: "

    formatted_data.append({"prompt": prompt, "response": output})

# Create dataset
dataset = Dataset.from_list(formatted_data)
print(f"✅ Data set created successfully, {len(dataset)} samples")


成功加载了52002条有效数据
使用前2000条数据进行训练
✅ 数据集创建成功，共2000个样本


In [40]:
dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 2000
})

In [42]:
dataset['response'][:3]

['1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'The three primary colors are red, blue, and yellow.',
 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.']

In [43]:
dataset['prompt'][:3]

['Instruction: Give three tips for staying healthy.\nOutput: ',
 'Instruction: What are the three primary colors?\nOutput: ',
 'Instruction: Describe the structure of an atom.\nOutput: ']

In [37]:
# 1. 加载Llama-3.2-1B模型
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=1024,
    dtype=torch.float16,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [38]:
# 2. 添加LoRA适配器
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.8 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [46]:
# 3. 设置训练参数
training_args = TrainingArguments(
    output_dir="models/llama-3.2-1b-instruction-unsloth",
    num_train_epochs=1,
    per_device_train_batch_size=4,  # 更小的模型，可以用更大的批量
    gradient_accumulation_steps=4,  # 可以减少累积步骤
    gradient_checkpointing=True,
    optim="adamw_torch",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    fp16=True,
    report_to="none",
)

In [50]:
# create formatting function - return a list of strings
def formatting_func(examples):
    """
    Formatting function - return a list of strings

    Args:
        examples: a dictionary containing 'prompt' and 'response' fields

    Returns:
        List: a list of processed strings
    """
    prompts = examples["prompt"]
    responses = examples["response"]

    # Ensure input is a list    
    if not isinstance(prompts, list):
        prompts = [prompts]
        responses = [responses]

    # build the format following Llama 3.2
    formatted_texts = []
    for prompt, response in zip(prompts, responses):
        # build the format following Llama 3.2
        formatted_text = f"<|system|>\nYou are a helpful, accurate and truthful AI assistant.\n<|user|>\n{prompt.strip()}\n<|assistant|>\n{response.strip()}"
        formatted_texts.append(formatted_text)

    return formatted_texts

In [51]:
# 5. 准备训练器
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,  # 使用已有数据集
    formatting_func=formatting_func,  # 添加格式化函数
    args=training_args,
    max_seq_length=1024,
    packing=False,
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [52]:
trainer

In [53]:
# 5. 显示训练信息
print("训练设置:")
print(f"- 模型: unsloth/Llama-3.2-1B-bnb-4bit")
print(f"- 数据集样本数: {len(dataset)}")
print(f"- 批量大小: {training_args.per_device_train_batch_size}")
print(f"- 梯度累积: {training_args.gradient_accumulation_steps}")
print(f"- 训练轮数: {training_args.num_train_epochs}")
print(f"- LoRA秩: 16")

训练设置:
- 模型: unsloth/Llama-3.2-1B-bnb-4bit
- 数据集样本数: 2000
- 批量大小: 4
- 梯度累积: 4
- 训练轮数: 1
- LoRA秩: 16


In [54]:
# 显示可训练参数数量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可训练参数: {trainable_params:,} ({100 * trainable_params / total_params:.4f}% 占总参数)")


可训练参数: 11,272,192 (1.4821% 占总参数)


In [55]:
# 6. 开始训练
print("\n🔄 开始训练...")
trainer.train()


🔄 开始训练...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/760,547,328 (1.48% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.322900
20,1.388300
30,1.277200
40,1.223200
50,1.216300
60,1.201900
70,1.224700
80,1.171400
90,1.180000
100,1.158400


TrainOutput(global_step=125, training_loss=1.2937623710632324, metrics={'train_runtime': 208.942, 'train_samples_per_second': 9.572, 'train_steps_per_second': 0.598, 'total_flos': 1855589126307840.0, 'train_loss': 1.2937623710632324})

In [56]:
# 7. 保存模型
output_dir = "models/llama-3.2-1b-instruction-unsloth/final"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ 训练完成，模型保存在: {output_dir}")

✅ 训练完成，模型保存在: models/llama-3.2-1b-instruction-unsloth/final


In [57]:
# @title 压缩并下载训练好的模型
from google.colab import files
import os
import shutil

# 模型路径
model_path = "models/llama-3.2-1b-instruction-unsloth/final"

# 检查模型文件夹是否存在
if os.path.exists(model_path):
    # 压缩模型文件夹
    output_file = "llama-3.2-1b-instruction-unsloth.zip"
    print(f"正在压缩模型文件夹到 {output_file}...")
    !zip -r {output_file} {model_path}

    # 下载压缩包
    print("开始下载模型压缩包...")
    files.download(output_file)
    print("下载已启动，请检查浏览器下载区域")
else:
    print(f"❌ 错误：模型目录 {model_path} 不存在")

正在压缩模型文件夹到 llama-3.2-1b-instruction-unsloth.zip...
  adding: models/llama-3.2-1b-instruction-unsloth/final/ (stored 0%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/README.md (deflated 66%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/tokenizer.json (deflated 85%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/tokenizer_config.json (deflated 96%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/adapter_model.safetensors (deflated 8%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/special_tokens_map.json (deflated 71%)
  adding: models/llama-3.2-1b-instruction-unsloth/final/adapter_config.json (deflated 56%)
开始下载模型压缩包...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

下载已启动，请检查浏览器下载区域


In [58]:
!pwd

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare


In [59]:
# 合并LoRA适配器与基础模型
from unsloth import FastLanguageModel
import torch

# 1. 加载基础模型
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",  # 基础模型路径
    max_seq_length=1024,
    dtype=torch.float16,
)

# 2. 加载您的LoRA适配器
# models/llama-3.2-1b-instruction-unsloth/final

# adapter_path = "models/llama-3.2-1b-instruction-unsloth/final"  # 替换为您下载的适配器路径
# model = FastLanguageModel.from_pretrained(
#     model_path=adapter_path,
#     model=base_model,  # 传入基础模型
#     tokenizer=tokenizer
# )

# # 3. 合并LoRA权重到基础模型中
# merged_model = FastLanguageModel.merge_lora_to_base_model(model)

# # 4. 保存完整的合并模型
# output_path = "llama-3.2-1b-merged"
# merged_model.save_pretrained(output_path)
# tokenizer.save_pretrained(output_path)

# print(f"✅ 模型已合并并保存到: {output_path}")

==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'model_path'

In [61]:
from peft import PeftModel
# 2. 加载您的LoRA适配器
adapter_path = "models/llama-3.2-1b-instruction-unsloth/final"  # 适配器路径
# 正确的方法是使用PeftModel.from_pretrained
model = PeftModel.from_pretrained(base_model, adapter_path)

In [62]:
# 3. 合并LoRA权重到基础模型
merged_model = model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [63]:
# 4. 保存完整的合并模型
output_path = "llama-3.2-1b-merged"
merged_model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"✅ 模型已合并并保存到: {output_path}")

✅ 模型已合并并保存到: llama-3.2-1b-merged


In [64]:
!pwd

/content/MSE718FinalProject-FinetuneAnalysisForLLMTechnique/llm-peft-compare


In [65]:
# 直接压缩并下载合并后的模型
from google.colab import files
import os

# 模型路径
model_path = "llama-3.2-1b-merged"

# 检查模型文件夹是否存在
if os.path.exists(model_path):
    # 查看模型大小（供参考）
    !du -sh {model_path}

    # 压缩模型文件夹
    output_file = "llama-3.2-1b-merged.zip"
    print(f"正在压缩模型到 {output_file}...")
    !zip -r {output_file} {model_path}

    # 下载压缩包
    print("开始下载模型压缩包...")
    files.download(output_file)
    print("下载已启动，请检查浏览器下载区域")
else:
    print(f"❌ 错误：模型目录 {model_path} 不存在")

997M	llama-3.2-1b-merged
正在压缩模型到 llama-3.2-1b-merged.zip...
  adding: llama-3.2-1b-merged/ (stored 0%)
  adding: llama-3.2-1b-merged/config.json (deflated 55%)
  adding: llama-3.2-1b-merged/tokenizer.json (deflated 85%)
  adding: llama-3.2-1b-merged/tokenizer_config.json (deflated 96%)
  adding: llama-3.2-1b-merged/generation_config.json (deflated 37%)
  adding: llama-3.2-1b-merged/special_tokens_map.json (deflated 71%)
  adding: llama-3.2-1b-merged/model.safetensors (deflated 13%)
开始下载模型压缩包...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

下载已启动，请检查浏览器下载区域
